# Защита персональных данных клиентов страховой компании "Хоть потоп"

**Заказчик:** страховая компания "Хоть потоп"

**Задача:** необходимо защитить данные клиентов страховой компании «Хоть потоп», разработав метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Важно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось
* **Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.
* **Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

pd.options.mode.chained_assignment = None

In [ ]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

In [ ]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [ ]:
df.shape

(5000, 5)

In [ ]:
# отклонений не наблюдается
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [ ]:
# пустые значения отсутствуют
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [ ]:
print("\033[1m" + 'Корреляция признаков' + "\033[0m" )
df.corr().round(2)

Корреляция признаков


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.00,0.00,0.01,-0.01,0.01
Возраст,0.00,1.00,-0.02,-0.01,0.65
Зарплата,0.01,-0.02,1.00,-0.03,-0.01
Члены семьи,-0.01,-0.01,-0.03,1.00,-0.04
Страховые выплаты,0.01,0.65,-0.01,-0.04,1.00


**Вывод:**

1) Удалены дубликаты (153 позиции)

2) Наибольшая положительная корреляция между страховыми случаями и возрастом, между остальными признаками не наблюдается сильной зависимости

## Умножение матриц

In [ ]:
# определение целевого признака и обучающих
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [ ]:
# прибавляем к обучающим признакам первую колонку из "1"
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
# определяем "w" - вектор весов линейной регрессии (нулевой элемент равен сдвигу), вычисляя по формуле минимизации MSE
w = np.linalg.inv(X.T @ X) @ X.T @ y
display(w[1:])
# вычислим значения линейной регрессии из scikit-learn и сравним с полученными по формуле
model = LinearRegression()
model.fit(features, target)
model.coef_
# значения совпадают

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

In [ ]:
# модель работает лучше среднего
predictions = model.predict(features)
print("R2_score -", r2_score(target, predictions))


R2_score - 0.4249455028666801


Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново)
    
a. Изменится. Приведите примеры матриц

b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной



$$
w = (X^T X)^{-1} X^T y
$$

$$
a = Xw = X(X^T X)^{-1} X^T y
$$

$$
a_p = XP((XP)^T(XP))^{-1}(XP)^Ty = 
$$
$$
= XP(P^T (X^T X) P)^{-1} P^T X^Ty =
$$
$$
= XPP^{-1} (X^T X)^{-1}(P^T)^{-1}P^T  X^T y
$$

$$
PP^{-1} = E (СОКРАЩАЕМ)
$$

$$
(P^T)^{-1}P^T = E (СОКРАЩАЕМ)
$$

$$
a_p = X(X^TX)^{-1}X^Ty
$$



Как видно, значение предсказания $a$ не меняется, если умножать матрицу признаков на обратимую матрицу.

## Алгоритм преобразования

**Алгоритм**

Таким образом, для защиты данных клиентов страховой компании «Хоть потоп» на этапе поиска коэффициентов регрессии и предсказаний регрессии будем умножать матрицу признаков на обратимую матрицу $Y$, которая будет генерироваться случайным образом.

Этапы алгоритма:
1. Составление матрицы $Y$.
2. Проверка матрицы на обратимость. Вычисление детерминанта матрицы $Y$.
3. Получение матрицы преобразованных признаков $Z = XY$.
4. Применение алгоритма на преобразованных признаках $Z$.

**Обоснование**

Матрица $Y$ должна иметь необходимую размерность $(nxn)$, где n - количество признаков для регрессии.
Таким образом, матрица $Z$ будет иметь туже размерность, что и матрица $X$.

Обратная матрица $Y$ существует только для квадратных невырожденных матриц (определитель которых отличен от нуля).


Таким образом, для защиты данных клиентов страховой компании «Хоть потоп» на этапе поиска коэффициентов регрессии и предсказаний регрессии будем умножать матрицу признаков на обратимую матрицу  $𝑌$, которая будет генерироваться случайным образом. Так как нам удалось доказать, что при умножении признаков на обратимую матрицу ничего не поменялось

$$
X*Y = Z
$$

$$
[5000x4]*[4x4]=[5000x4]
$$

Таким образом, обратимая матрица $Y$ должна быть размерностью $[4х4]$ 

## Проверка алгоритма

In [ ]:
# делим выборку на тестовую и обучающую
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

**Проверяем качество модели на чистых данных, отмасштабированных и нормализованных**

In [ ]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = (np.linalg.inv(X.T@X))@X.T@y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print("R2_score чистых данных - ", r2_score(target_test, predictions))

R2_score чистых данных -  0.4352275712702667


In [ ]:
scaler = StandardScaler()
model = LinearRegression()
scaler.fit(features_train)
features_train_scaled = scaler.transform(features_train)
features_test_scaled = scaler.transform(features_test)
model.fit(features_train_scaled, target_train)
predictions_scaled = model.predict(features_test_scaled)
print("R2_score отмасштабированных данных - ", r2_score(target_test, predictions_scaled))


R2_score отмасштабированных данных -  0.4352275712702667


In [ ]:
scaler2 = MinMaxScaler()
columns = features_train.columns

scaler2.fit(features_train)
features_train_nor = scaler2.transform(features_train)
features_test_nor = scaler2.transform(features_test)
features_train_norm = pd.DataFrame(features_train_nor, columns=columns)
features_test_norm = pd.DataFrame(features_test_nor, columns=columns)

model = LinearRegression()
model.fit(features_train_norm, target_train)
predictions_norm = model.predict(features_test_norm)
print("R2_score нормализованных данных - ", r2_score(target_test, predictions_norm))

R2_score нормализованных данных -  0.43522757127026646


In [ ]:
scaler = StandardScaler()
model = LinearRegression()
scaler.fit(features_train_norm)
features_train_scaled2 = scaler.transform(features_train_norm)
features_test_scaled2 = scaler.transform(features_test_norm)
model.fit(features_train_scaled2, target_train)
predictions_scaled2 = model.predict(features_test_scaled2)
print("R2_score нормализованных и отмасштабированных данных - ", r2_score(target_test, predictions_scaled2))


R2_score нормализованных и отмасштабированных данных -  0.4352275712702667


**Вывод:**

r2_score на отмасштабированных и нормализованных данных чуть выше

**Функция преобразования признаков**  Z=X*Y

In [ ]:
n = features.shape[1]
np.random.seed(12345)
Y_matrix = np.random.randint(1, 10, (n,n))
print('Y_matrix')
print(Y_matrix)
print()
print('DET =', np.linalg.det(Y_matrix)) # не равен 0
print()
Z_matrix = features @ Y_matrix # X*Y
print('Z_matrix')
print (Z_matrix)

Y_matrix
[[3 6 2 5]
 [6 3 2 7]
 [2 8 7 1]
 [3 2 3 7]]

DET = -616.9999999999995

Z_matrix
             0         1         2        3
0      99452.0  396931.0  347287.0  49899.0
1      76279.0  304140.0  266095.0  38329.0
2      42174.0  168087.0  147058.0  21203.0
3      83532.0  333667.0  291948.0  41861.0
4      52371.0  208890.0  182758.0  26301.0
...        ...       ...       ...      ...
4995   71574.0  285688.0  249962.0  35910.0
4996  105007.0  419304.0  366871.0  52645.0
4997   67926.0  271264.0  237346.0  34054.0
4998   65544.0  261678.0  228955.0  32880.0
4999   81374.0  324892.0  284261.0  40808.0

[5000 rows x 4 columns]


In [ ]:
display(features.head()) # признаки до преобразования X
display()
display(Z_matrix.head()) # признаки после преобразования Z
display(Y_matrix) # обратная матрица признаков Y


,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


,0,1,2,3
0,99452.0,396931.0,347287.0,49899.0
1,76279.0,304140.0,266095.0,38329.0
2,42174.0,168087.0,147058.0,21203.0
3,83532.0,333667.0,291948.0,41861.0
4,52371.0,208890.0,182758.0,26301.0


array([[3, 6, 2, 5],
       [6, 3, 2, 7],
       [2, 8, 7, 1],
       [3, 2, 3, 7]])

In [ ]:
# деление на обучающуюю и тренировочную выборку зашифрованных данных (Z)
features_train, features_test, target_train, target_test = train_test_split(
    Z_matrix, target, test_size=0.25, random_state=12345)

In [ ]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print("R2_score чистых данных - ", r2_score(target_test, predictions))

R2_score чистых данных -  0.43522758162636566


In [ ]:
scaler2 = MinMaxScaler()
columns = features_train.columns

scaler2.fit(features_train)
features_train_nor = scaler2.transform(features_train)
features_test_nor = scaler2.transform(features_test)
features_train_norm = pd.DataFrame(features_train_nor, columns=columns)
features_test_norm = pd.DataFrame(features_test_nor, columns=columns)

model = LinearRegression()
model.fit(features_train_norm, target_train)
predictions_norm = model.predict(features_test_norm)
print("R2_score нормализованных данных - ", r2_score(target_test, predictions_norm))

R2_score нормализованных данных -  0.435227566864531


In [ ]:
scaler = StandardScaler()
model = LinearRegression()
scaler.fit(features_train)
features_train_scaled = scaler.transform(features_train)
features_test_scaled = scaler.transform(features_test)

model.fit(features_train_scaled, target_train)
predictions_scaled = model.predict(features_test_scaled)
print("R2_score отмасштабированных данных - ", r2_score(target_test, predictions_scaled))

R2_score отмасштабированных данных -  0.43522757291743375


In [ ]:
scaler = StandardScaler()
model = LinearRegression()
scaler.fit(features_train_norm)
features_train_scaled2 = scaler.transform(features_train_norm)
features_test_scaled2 = scaler.transform(features_test_norm)

model.fit(features_train_scaled2, target_train)
predictions_scaled2 = model.predict(features_test_scaled2)
print("R2_score нормализованных и отмасштабированных данных - ", r2_score(target_test, predictions_scaled2))
print("MSE - ", mean_squared_error(target_test, predictions_scaled2))


R2_score нормализованных и отмасштабированных данных -  0.43522757295903935
MSE -  0.11660517437658488


**Вывод:**

r2_score не изменился, mse близок к 0

**Дешифровываем данные**

In [ ]:
# получаем X (Z*(Y)**(-1))
original_X = Z_matrix @ np.linalg.inv(Y_matrix)
target_test = pd.DataFrame(target_test)
total_test_df = pd.DataFrame(original_X, index = target_test.index, columns=Z_matrix.columns).join(target_test)
# изменяем формат экспоненты
total_test_df[0]=total_test_df[0].apply(lambda x: '%.3f' % x)
total_test_df[3]=total_test_df[3].apply(lambda x: '%.3f' % x)
total_test_df

,0,1,2,3,Страховые выплаты
3183,-0.000,33.0,39000.0,4.000,0
1071,-0.000,50.0,43100.0,2.000,2
2640,1.000,39.0,42100.0,0.000,0
2282,-0.000,20.0,34800.0,-0.000,0
1595,-0.000,41.0,40000.0,4.000,0
...,...,...,...,...,...
982,1.000,51.0,29000.0,2.000,2
3820,1.000,33.0,57900.0,3.000,0
3595,1.000,35.0,42300.0,-0.000,0
3513,1.000,36.0,41300.0,-0.000,0


In [ ]:
predictions_df = pd.DataFrame(predictions_scaled2, index = target_test.index)

In [ ]:
# функция для сдвига
predict = []
for i in predictions_df[0]:
    if i > 0.5:
        predict.append('1')
    else:
        predict.append('0')

In [ ]:
# предсказания со сдвигом
total_test_df['Предсказания'] = predict
total_test_df

,0,1,2,3,Страховые выплаты,Предсказания
3183,-0.000,33.0,39000.0,4.000,0,0
1071,-0.000,50.0,43100.0,2.000,2,1
2640,1.000,39.0,42100.0,0.000,0,0
2282,-0.000,20.0,34800.0,-0.000,0,0
1595,-0.000,41.0,40000.0,4.000,0,0
...,...,...,...,...,...,...
982,1.000,51.0,29000.0,2.000,2,1
3820,1.000,33.0,57900.0,3.000,0,0
3595,1.000,35.0,42300.0,-0.000,0,0
3513,1.000,36.0,41300.0,-0.000,0,0


In [ ]:
# полная таблица - с сходными данными, дешифрованными (_дш), целевым признаком и предсказзанием
total_df =pd.DataFrame(features, index = total_test_df.index, columns=features.columns).join(total_test_df)
total_df=total_df.rename(columns={0:"Пол_дш", 1:"Возраст_дш", 2:"Зарплата_дш", 3:"Члены семьи_дш"})
total_df

,Пол,Возраст,Зарплата,Члены семьи,Пол_дш,Возраст_дш,Зарплата_дш,Члены семьи_дш,Страховые выплаты,Предсказания
3183,0,33.0,39000.0,4,-0.000,33.0,39000.0,4.000,0,0
1071,0,50.0,43100.0,2,-0.000,50.0,43100.0,2.000,2,1
2640,1,39.0,42100.0,0,1.000,39.0,42100.0,0.000,0,0
2282,0,20.0,34800.0,0,-0.000,20.0,34800.0,-0.000,0,0
1595,0,41.0,40000.0,4,-0.000,41.0,40000.0,4.000,0,0
...,...,...,...,...,...,...,...,...,...,...
982,1,51.0,29000.0,2,1.000,51.0,29000.0,2.000,2,1
3820,1,33.0,57900.0,3,1.000,33.0,57900.0,3.000,0,0
3595,1,35.0,42300.0,0,1.000,35.0,42300.0,-0.000,0,0
3513,1,36.0,41300.0,0,1.000,36.0,41300.0,-0.000,0,0


**Общий вывод:**

Качество регрессии не снизилось на зашифрованных данных, значение не меняется на изначальных и зашифрованных данных (r2_score осталось 42,3% )

Исходные значения легко восстанавливаются, имея нужный ключ (cipher_matrix)

